In [5]:
from vdom import *
from vdom.svg import *
from functools import reduce

In [1]:
def generate_events(length=30):
    events = [25]
    for x in range(length):
        events.append(generate_next_event(events))
    return events

In [2]:
def generate_next_event(events):
    return min(max(events[-1] + random.randint(-3, 3), 0), 119)

In [6]:
import random
import time

def minichart(lines, title=""):
    # if lines is a 1D array, we'll assume an equidistant x array
    # TODO: Scale
    height = 120
    
    xscale = 30

    points = reduce(lambda s, coords: f"{s}{coords[0]*xscale},{height - coords[1]} ", enumerate(lines), "")
    
    last_point = ((len(lines) - 1)*xscale, height - lines[-1])
    
    return svg(
        polyline(fill="none", stroke="black", points=points),
        circle(cx=str(last_point[0]), cy=str(last_point[1]), r="2", fill="red"),
        text(title, x="0", y="20", fontWeight="600", fontFamily="Verdana"),
        width="920", height="120", style={ 'borderTop': '1px solid #f7f7f7', 'paddingTop': '20px'}
    )

In [7]:
events = generate_events()

handle = display(minichart(events), display_id=True)

<svg />

In [8]:
for x in range(300):
    time.sleep(30/1000.)
    events.pop(0)
    events.append(generate_next_event(events))
    handle.update(minichart(events))

In [9]:
import psutil
psutil.cpu_percent()

0.0

In [10]:
psutil.virtual_memory().percent

86.2

In [11]:
cpu_percents = [psutil.cpu_percent()]
vmem_percents = [psutil.virtual_memory().percent]

In [12]:
def append_cpu_percent(events, max_length=30):
    events.append(psutil.cpu_percent())
    if(len(events) > max_length):
        events.pop(0)
    return events

def append_virtual_memory_percent(events, max_length=30):
    events.append(psutil.virtual_memory().percent)
    if(len(events) > max_length):
        events.pop(0)
    return events

In [13]:
def system_metrics(cpu_percents, vmem_percents):
    return div(
        minichart(cpu_percents, title=f"CPU {cpu_percents[-1]}%"),
        br(), # lazy padding
        minichart(vmem_percents, title=f"Memory {vmem_percents[-1]}%"),
    )
    

sys_handle = display(system_metrics(cpu_percents, vmem_percents), display_id=True)

for x in range(300):
    time.sleep(100/1000.)
    append_cpu_percent(cpu_percents)
    append_virtual_memory_percent(vmem_percents)
    
    sys_handle.update(system_metrics(cpu_percents, vmem_percents))

<div />